# Three Concentric Circles Demo

Simple demonstration of the spectral clustering algorithm on three concentric circles.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '..')

from fitkit.community import SpectralCluster

%matplotlib inline

In [ ]:
# Generate three concentric circles (30 points per circle = 90 total)
np.random.seed(1)
npts = 30
step = 2*np.pi/npts
theta = np.arange(step, 2*np.pi + step, step)
radius = np.random.randn(npts)

r1 = np.ones(npts) + 0.1*radius
r2 = 2*np.ones(npts) + 0.1*np.random.randn(npts)
r3 = 3*np.ones(npts) + 0.1*np.random.randn(npts)

circle1 = np.column_stack([r1*np.cos(theta), r1*np.sin(theta)])
circle2 = np.column_stack([r2*np.cos(theta), r2*np.sin(theta)])
circle3 = np.column_stack([r3*np.cos(theta), r3*np.sin(theta)])

X = np.vstack([circle1, circle2, circle3])
true_labels = np.array([0]*npts + [1]*npts + [2]*npts)

print(f"Dataset shape: {X.shape}")

# Plot the data
plt.figure(figsize=(8, 8))
plt.scatter(X[:, 0], X[:, 1], c=true_labels, cmap='tab10', s=30, alpha=0.7)
plt.title('Three Concentric Circles (True Labels)')
plt.axis('equal')
plt.colorbar(label='True Cluster')
plt.show()

In [ ]:
# Run spectral clustering
# NOTE: Sigma needs to scale with dataset size/density:
#   - 300 points (100/circle): sigma=0.158 works well (MATLAB equivalent)
#   - 150 points (50/circle): sigma=0.2 works well
#   - 90 points (30/circle): sigma=0.2 works well
# For this demo with 30 points/circle, use sigma=0.2
sigma = 0.2

print(f"Running SpectralCluster with sigma={sigma:.4f}...")
clf = SpectralCluster(sigma=sigma, lambda_=0.2, max_clusters=10, verbose=True)
clf.fit(X)

print(f"\n✓ Found {clf.n_clusters_} clusters")
print(f"Label distribution: {np.bincount(clf.labels_)}")

In [ ]:
# Plot the detected clusters
plt.figure(figsize=(8, 8))
plt.scatter(X[:, 0], X[:, 1], c=clf.labels_, cmap='tab10', s=30, alpha=0.7)
plt.title(f'Detected Clusters (n={clf.n_clusters_})')
plt.axis('equal')
plt.colorbar(label='Detected Cluster')
plt.show()

# Check if we found the correct number
if clf.n_clusters_ == 3:
    print("✓ SUCCESS: Correctly detected 3 clusters!")
else:
    print(f"Expected 3 clusters, found {clf.n_clusters_}")

In [ ]:
# Compute cluster purity (how well detected clusters match true labels)
for i in range(clf.n_clusters_):
    mask = clf.labels_ == i
    true_in_cluster = true_labels[mask]
    if len(true_in_cluster) > 0:
        most_common = np.bincount(true_in_cluster).argmax()
        purity = (true_in_cluster == most_common).sum() / len(true_in_cluster)
        print(f"Cluster {i}: {mask.sum()} points, purity={purity:.2%}")